In [39]:
import pandas as pd 
data_17_18 = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/DatasetsOriginais/transfermarkt/transfermarkt_fbref_201718.csv', delimiter = ';',  index_col=0)
data_18_19 = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/DatasetsOriginais/transfermarkt/transfermarkt_fbref_201819.csv', delimiter = ';',  index_col=0,  low_memory=False)
data_19_20 = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/DatasetsOriginais/transfermarkt/transfermarkt_fbref_201920.csv', delimiter = ';',  index_col=0)


In [40]:
# Concatenate the DataFrames vertically
data = pd.concat([data_17_18, data_18_19, data_19_20], ignore_index=True)

In [41]:
data_17_18['season_year'] = 2018
data_18_19['season_year'] = 2019
data_19_20['season_year'] = 2020

In [42]:
data = data.drop('Season', axis=1)
data = data.drop('Attendance', axis=1)


# Extract country code from the nationality column
data['nationality'] = data['nationality'].str[-3:]

data = data.dropna()

null_counts = data.isnull().sum()
print(null_counts)

data.shape[0]

player          0
nationality     0
position        0
squad           0
age             0
               ..
xGDiff          0
xGDiff/90       0
CL              0
WinCL           0
CLBestScorer    0
Length: 397, dtype: int64


6409

In [43]:
#Criar os bins

import pandas as pd
from sklearn.preprocessing import LabelEncoder


# Defina os limites dos intervalos
limites = [0, 5e6, 10e6, 15e6, 20e6, 25e6, 50e6, 75e6, 100e6, float('inf')]

# Defina os rótulos dos intervalos
rotulos = ['1 - 0-5', '2 - 5-10', '3 - 10-15', '4 - 15-20', '5 - 20-25', '6 - 25-50', '7 - 50-75', '8 - 75-100', '9 - 100+']

# Crie a nova coluna "bins" usando pd.cut()
data['bins'] = pd.cut(data['value'], bins=limites, labels=rotulos)

# Converta a coluna "bins" em uma variável categórica
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['bins'])

# Exiba o DataFrame resultante
print(data)

# DEBUG

limite = 125e6  # Limite de valor

# Encontrar a linha que atende aos critérios
linha = data.loc[data['value'] > limite]

# Imprimir a linha encontrada
print(linha)




              player nationality position        squad   age  birth_year  \
0             Burgui         ESP    FW,MF       Alavés  23.0      1993.0   
1     Raphaël Varane         FRA       DF  Real Madrid  24.0      1993.0   
2       Rubén Duarte         ESP       DF       Alavés  21.0      1995.0   
3      Samuel Umtiti         FRA       DF    Barcelona  23.0      1993.0   
4        Manu García         ESP       MF       Alavés  31.0      1986.0   
...              ...         ...      ...          ...   ...         ...   
6691   Ryan Bertrand         ENG       DF  Southampton  29.0      1989.0   
6692  Sofiane Boufal         MAR    MF,FW  Southampton  25.0      1993.0   
6693     Kevin Danso         AUT       DF  Southampton  20.0      1998.0   
6694  Moussa Djenepo         MLI    MF,DF  Southampton  21.0      1998.0   
6695      Angus Gunn         ENG       GK  Southampton  23.0      1996.0   

           value  height                      position2   foot  ... Pts/G  \
0      180

In [44]:
# correlação

# Selecione apenas as colunas numéricas

colunas_numericas = data.select_dtypes(include=[float, int])

# Calcule a correlação entre as variáveis numéricas e a variável alvo
correlacao = colunas_numericas.corrwith(data['value'])

# Criar uma lista para armazenar as colunas a serem removidas
colunas_removidas = []

for coluna in colunas_numericas.columns:
    correlacao_valor = correlacao[coluna]
    print(f"Coluna: {coluna}, Correlação: {correlacao_valor}")
    
    # Verificar se a correlação é negativa
    if correlacao_valor < 0:
        colunas_removidas.append(coluna)

# Remover as colunas com correlação negativa do DataFrame
data = data.drop(colunas_removidas, axis=1)

data.to_csv("merged_class.csv", index = False)

Coluna: age, Correlação: -0.10924859809452325
Coluna: birth_year, Correlação: 0.024854423321180263
Coluna: value, Correlação: 1.0
Coluna: height, Correlação: -0.012926637029129977
Coluna: games, Correlação: 0.3567899602802516
Coluna: games_starts, Correlação: 0.3599651823953477
Coluna: minutes, Correlação: 0.35889879565034566
Coluna: goals, Correlação: 0.5164508021942003
Coluna: assists, Correlação: 0.4770254630210917
Coluna: pens_made, Correlação: 0.23626015549299031
Coluna: pens_att, Correlação: 0.24352762733914832
Coluna: cards_yellow, Correlação: 0.12566504790960162
Coluna: cards_red, Correlação: 0.028575486790501408
Coluna: goals_per90, Correlação: 0.15517156460197298
Coluna: assists_per90, Correlação: 0.2499113506066444
Coluna: goals_assists_per90, Correlação: 0.21994192930534043
Coluna: goals_pens_per90, Correlação: 0.14297407290199302
Coluna: goals_assists_pens_per90, Correlação: 0.210383631781622
Coluna: xg, Correlação: 0.4908361672112905
Coluna: npxg, Correlação: 0.5030875150

In [45]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error

dataset = pd.read_csv('/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/Scripts and Notebooks/merged_class.csv')

dropped = dataset.drop(['value','bins','target'], axis=1)

# Select the relevant features and target variable
features_scale = dropped.drop(['player', 'nationality', 'position', 'squad', 'position2', 'foot', 'league'], axis=1)
features_encode = dropped[[ 'player', 'nationality', 'position', 'squad', 'position2', 'foot', 'league']]
target = dataset['target']

# Preprocess the scaled features
scaled_features = features_scale.apply(pd.to_numeric, errors='coerce').values
scaled_features = np.nan_to_num(scaled_features)

# One-hot encode the categorical features
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(features_encode).toarray()

# Concatenate the scaled features and encoded features
features = np.concatenate((scaled_features, encoded_features), axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Criar o modelo de regressão logística
model = LogisticRegression(max_iter=1000,solver='liblinear')

# Treinar o modelo com os dados de treinamento
model.fit(X_train, y_train)

# Fazer previsões com os dados de teste
y_pred = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: {:.2f}%".format(accuracy * 100))

accuracy: 61.93%
